In [1]:
%pwd

'/content'

In [2]:
# For use in google Collab, run this cell once, and it will copy the data into the collab and move to the right dir.
!git clone https://github.com/fuentediezjaime/gas_detection_challenge
%pwd
%cd gas_detection_challenge/notebooks/

Cloning into 'gas_detection_challenge'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 82 (delta 3), reused 0 (delta 0), pack-reused 74 (from 1)
Receiving objects: 100% (82/82), 68.79 MiB | 6.83 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Updating files: 100% (31/31), done.
/content/gas_detection_challenge/notebooks


In [1]:
#extend path for loading modules
import sys
import os
sys.path.append(os.path.abspath( '../src/models'))
##from custom_functions import loss_custom, hyperpar_search
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import numpy as np
import lightgbm as lgb

In [ ]:
x_train_total = pd.read_csv('../data/raw/x_train_T9QMMVq.csv')
y_train_total = pd.read_csv('../data/raw/y_train_R0MqWmu.csv')

x_train, x_test, y_train, y_test = train_test_split(x_train_total, y_train_total, test_size=0.2, random_state=22)

x_train = x_train.drop(columns='ID',axis=1)
y_train = y_train.drop(columns='ID',axis=1)

x_test = x_test.drop(columns='ID',axis=1)
y_test = y_test.drop(columns='ID',axis=1)



In [ ]:
def custom_error(estimator, x_test, y_true, **kwargs):
    """
    Calcula el error cuadrático medio personalizado para el regresor multiobjetivo (vectorizado).

    Args:
        y_true (np.ndarray): Valores verdaderos, forma (n_muestras, 24).
        y_pred (np.ndarray): Valores predichos, forma (n_muestras, 24).

    Returns:
        float: Error cuadrático medio personalizado promedio.
    """

    y_pred = estimator.predict(x_test)
    scale_factors = np.where(y_true < 0.5, 1.0, 1.2)  # Factor de escala
    squared_errors = (y_true - y_pred)**2
    weighted_errors = scale_factors * squared_errors
    return np.mean(weighted_errors)

In [ ]:
# Define my model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
import time
from sklearn.metrics import make_scorer
import joblib


#model = RandomForestRegressor()
#model_lgbm = LGBMRegressor(n_jobs=-1)
# Set hyperparameter range
param_dist = {
    'estimator__n_estimators': [10,50,100],  # Lista discreta de valores
    'estimator__max_depth': [10, 20, 30, 40, None],  # Valores posibles incluyendo None
    'estimator__min_samples_split': range(2, 20),  # Rango de 2 a 19
    'estimator__min_samples_leaf': range(1, 10)}   # Rango de 1 a 9
#    'max_features': ['sqrt', 'log2', None]  # Estrategias de selección de características
#}

scorer =  make_scorer(custom_error, greater_is_better=False)

lgbm = lgb.LGBMRegressor(random_state=42, n_jobs=-1,verbose=-1,device='gpu')
multioutput_lgbm = MultiOutputRegressor(lgbm)
rand_search = RandomizedSearchCV(estimator=multioutput_lgbm,
                                 param_distributions=param_dist,
                                cv=2,n_iter=20,n_jobs=14,scoring=scorer,verbose=1)


#start_fit = time.perf_counter()
#rand_search.fit(x_train, y_train)
#end_fit = time.perf_counter()
#print(f'El fit de RandomForestRegressor tarda {end_fit-start_fit} segundos')


start_fit = time.perf_counter()
rand_search.fit(x_train,y_train)
end_fit = time.perf_counter()
#print(f'El fit de LGBM  tarda {end_fit-start_fit} segundos')

joblib.dump(rand_search,'random_search_randforest.pkl')


Fitting 2 folds for each of 20 candidates, totalling 40 fits


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import cross_validate
best_model = rand_search.best_estimator_
print(rand_search.best_params_)

{'n_estimators': 25, 'max_depth': 10}


In [ ]:
best_model.fit(x_train,y_train)

RandomForestRegressor(max_depth=10, n_estimators=25, n_jobs=-1)

In [ ]:
cval = cross_validate(best_model, x_test, y_test, cv=5)

In [ ]:
print(cval)

{'fit_time': array([1.38574314, 1.41716647, 1.3673737 , 1.40675449, 1.43548656]), 'score_time': array([0.04904199, 0.0353024 , 0.03423285, 0.03337717, 0.03457189]), 'test_score': array([0.88220567, 0.87428545, 0.88148099, 0.88574643, 0.8570012 ])}
